<a href="https://colab.research.google.com/github/BrunoFoggiatto/Pibic-Visao_Computacional/blob/Modelos_Treinamento/C%C3%B3pia_de_2025_03_31_pibic6_D_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.5/983.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [3]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 13.6 MB/s eta 0:00:00


In [49]:
import easyocr
import os
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import glob
import re

In [50]:
# Carregar modelo e OCR
modelo = YOLO("/content/drive/MyDrive/Colab Notebooks/Yolo/treinamento_yolo11n5/weights/best.pt")
ocr = easyocr.Reader(['en'], gpu=True)


In [56]:
# Função para processar OCR e corrigir placas
def processar_placa(imagem):
    # Obter texto do OCR
    resultados_ocr = ocr.readtext(imagem)

    # Extrair todo o texto reconhecido
    texto_bruto = ''.join([texto.replace(" ", "").upper() for _, texto, _ in resultados_ocr])

    if not texto_bruto:
        return "_______", texto_bruto

    # Corrigir simbolos
    simbolos = {'~':'Q', '@':'Q', ']':'1', '?':'9', '/':'1', '\\':'1', '|':'1', ':':'8', ';':'3', '[':'1', '(':'1'}
    texto = ''.join([simbolos.get(c, c) for c in texto_bruto])

    texto = ''.join(c for c in texto if c.isalnum()).upper()

    # Verificar comprimento
    if len(texto) < 5:  # Muito curto para ser uma placa válida
        return "_______", texto_bruto

    # Se tiver mais de 7 caracteres, pegar os mais prováveis
    if len(texto) > 7:
        # Tentar encontrar padrão de placa (3 letras + 4 números ou 3 letras + 1 número + 1 letra/número + 2 números)
        padrao1 = re.search(r'[A-Z]{3}\d{4}', texto)
        padrao2 = re.search(r'[A-Z]{3}\d[A-Z0-9]\d{2}', texto)

        if padrao1:
            texto = padrao1.group(0)
        elif padrao2:
            texto = padrao2.group(0)
        else:
            # Se não encontrar padrão, pegar os últimos 7 caracteres
            texto = texto[-7:]

    # Completar para 7 caracteres se necessário
    if len(texto) < 7:
        return "_______", texto_bruto

    resultado = list(texto)

    # Sempre letras nas três primeiras posições
    for i in range(3):
        if not resultado[i].isalpha() or resultado[i] in '0123456789':
            # Substituir dígitos por letras correspondentes
            mapa_digito_para_letra = {
                '0': 'O',
                '1': 'I',
                '2': 'Z',
                '3': 'E',
                '4': 'A',
                '5': 'S',
                '6': 'G',
                '7': 'T',
                '8': 'B',
                '9': 'P'
            }
            resultado[i] = mapa_digito_para_letra.get(resultado[i], 'X') fallback X 

    # Sempre digito quarta posição
    if not resultado[3].isdigit():
        # Substituir letras por dígitos
        mapa_letra_para_digito = {
            'O': '0', 'Q': '0', 'D': '0',
            'I': '1', 'L': '1', 'J': '1',
            'Z': '2', 'S': '5', 'B': '8',
            'G': '6', 'T': '7', 'A': '4',
            'P': '9', 'E': '3'
        }
        resultado[3] = mapa_letra_para_digito.get(resultado[3], '0')

    # Posição 4 pode ser letra ou digito

    # Quinta e sexta posição sempre digito
    for i in [5, 6]:
        if not resultado[i].isdigit():
            # Substituir letras por dígitos
            mapa_letra_para_digito = {
                'O': '0', 'Q': '0', 'D': '0',
                'I': '1', 'L': '1', 'J': '1',
                'Z': '2', 'S': '5', 'B': '8',
                'G': '6', 'T': '7', 'A': '4',
                'P': '9', 'E': '3'
            }
            resultado[i] = mapa_letra_para_digito.get(resultado[i], '0')

    return ''.join(resultado), texto_bruto


In [37]:
# Função para melhorar a imagem antes do OCR
def pre_processar_imagem(imagem):
    # Converter para escala de cinza
    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

    # Redimensionar para melhorar resolução
    gray = cv2.resize(gray, (0, 0), fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

    # Melhorar o contraste
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)

    blur = cv2.bilateralFilter(gray, 11, 17, 17)

    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY_INV, 13, 2)

    # Melhorar a forma dos caracteres
    kernel = np.ones((2, 2), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    return thresh


In [53]:
def reconhecer_placas(pasta_teste):
    imagens = glob.glob(os.path.join(pasta_teste, "*.jpg"))
    resultados = []

    for img_path in imagens:
        nome_base = os.path.splitext(os.path.basename(img_path))[0]
        txt_path = os.path.join(pasta_teste, nome_base + ".txt")

        # Ler placa real
        placa_real = ""
        try:
            with open(txt_path, "r") as f:
                for linha in f:
                    if linha.lower().startswith("plate:"):
                        placa_real = linha.split("plate:")[1].strip().upper()
        except:
            continue

        if len(placa_real) != 7:
            continue

        # Ler imagem
        img = cv2.imread(img_path)
        if img is None:
            resultados.append((nome_base, "_______", placa_real, False, None, img_path, ""))
            continue

        # Yolo para detectar placa
        placa_pred = "_______"
        texto_bruto = ""
        bbox_placa = None

        deteccoes = modelo(img)
        for r in deteccoes:
            boxes = r.boxes
            if len(boxes) > 0:
                box = boxes[0]
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                bbox_placa = (x1, y1, x2, y2)
                roi = img[y1:y2, x1:x2]

                placa_pred, texto_bruto = processar_placa(roi)

                # Tentar com pré-processamento se falhar
                if placa_pred == "_______":
                    roi_processada = pre_processar_imagem(roi)
                    placa_pred_proc, texto_bruto_proc = processar_placa(roi_processada)
                    if placa_pred_proc != "_______":
                        placa_pred = placa_pred_proc
                        texto_bruto += " | " + texto_bruto_proc
                break

        # Verificar acerto
        acerto = placas_equivalentes(placa_pred, placa_real)
        resultados.append((nome_base, placa_pred, placa_real, acerto, bbox_placa, img_path, texto_bruto))

    return resultados


In [54]:
# Função para exibir resultados
def exibir_resultados(resultados):

    # Calcular taxa de reconhecimento
    acertos = sum(1 for _, pred, real, acerto, _, _, _ in resultados if acerto)
    total = len(resultados)
    taxa = (acertos / total) * 100 if total > 0 else 0

    print(f"\nTaxa de reconhecimento = {taxa:.2f}% [{acertos}/{total}]")

    # Erros
    if acertos < total:
        print("\nERROS:")
        for nome, pred, real, acerto, _, _, texto in resultados:
            if not acerto:
                texto_reconhecido = f" | OCR reconheceu: '{texto}'" if pred == "_______" else ""
                print(f"  - {nome}: Placa predita: '{pred}' | Placa real: '{real}'{texto_reconhecido}")

    return taxa, acertos, total


In [55]:
#Processamento
pasta_teste = "/content/drive/MyDrive/Colab Notebooks/DATASET_250/test"
resultados = reconhecer_placas(pasta_teste)
taxa, acertos, total = exibir_resultados(resultados)
print(f"Taxa de reconhecimento = {taxa:.2f}% [{acertos}/{total}]")


0: 384x640 1 class_0, 141.9ms
Speed: 4.9ms preprocess, 141.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 class_0, 124.1ms
Speed: 3.6ms preprocess, 124.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 class_0, 131.6ms
Speed: 3.3ms preprocess, 131.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 class_0, 125.7ms
Speed: 3.6ms preprocess, 125.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 class_0, 124.5ms
Speed: 3.1ms preprocess, 124.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 class_0, 136.9ms
Speed: 3.9ms preprocess, 136.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 class_0, 124.4ms
Speed: 3.8ms preprocess, 124.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 class_0, 124.1ms
Speed: 4.5ms preprocess, 124.1ms inference, 1.1ms postprocess per 